In [17]:
import re
from enum import Enum, auto
from typing import List
from functools import reduce

In [18]:

test_str = [""""this is a test,", 2.2, True, None, 2""",""""lass mal schauen","was geht,,,,)",2.12, None"""]

def value_split(str_list):
    r = r'(".+"|\d+\.\d+|True|False|None|\d+)'
    return list(re.findall(r,str_list))

for test in test_str:
    for i in value_split(test):print(i)


"this is a test,"
2.2
True
None
2
"lass mal schauen","was geht,,,,)"
2.12
None


In [19]:
test = "testing {price}"
l = {"price":2, "no":2}
test.format(**l)

'testing 2'

In [20]:
class DataTypes(Enum):
    INT = auto()
    FLOAT = auto()
    STRING = auto()
    BOOL = auto()
    NONE = auto()
    EXPR = auto()
    ATTR = auto()
    ATTRMAP = auto()


class FunctionBuilder():

    def __init__(self):
        self.regex_lookUp = {
            DataTypes.INT: "-?\d+",
            DataTypes.FLOAT:"-?\d+\.\d+",
            DataTypes.STRING:'"[^"]+"',
            DataTypes.BOOL: "True|False",
            DataTypes.NONE: "None",
            DataTypes.EXPR: ".+",
            DataTypes.ATTR: "`[^`]+`",
            DataTypes.ATTRMAP: """(?:`[^`]+`)=(?:-?\d+\.\d+|-?\d+|"[^"]+"|True|False|None)"""
        }

    def build_function(self, needed:dict,*args):
        """builds an regex for an function call defined by a dict

        Args:
            needed (dict): {
                text:str --> a text with or without marked spots to place regex for parameter finding e.g. "this is a function {parameter}" -- required
                types:dict a dictonary that maps paramter(defined in text) to the enum DataTypes which the parameter should have -- required
                repeating: dict maps parameters to boolean if there are of an variable lenght -- optional
                implicite: Bool a boolean parameter can be representet as text itself instead of a parameter --optional
            }
            *args if you want to extend the call with optional parameters --> same as needed

        Returns:
            _type_: _description_
        """
        whole_function, group_mapper, counter = self.unpacker(needed, 1)
        for arg in args:
            apendy = self.unpacker(arg, counter, True)
            whole_function += apendy[0]
            group_mapper.extend(apendy[1])
            counter = apendy[2]

        
        return (whole_function, group_mapper)

    def unpacker(self, function_part:dict, counter:int, optional:bool = False) -> tuple:
        func_regex = function_part.get("text")

        if function_part.get("implicite", False):
            if not optional:
                return (func_regex, [
                    {
                "regex":None,
                "group":None,
                "repeating":False,
                "needed": not optional,
                "implicite":True
            }
                ], counter)
            
            return (f"({func_regex})?", [{
                "regex":None,
                "group":counter,
                "repeating":False,
                "needed": not optional,
                "implicite":True
            }],counter+1)
        types = function_part.get("types")
        repeating = function_part.get("types", dict())
        parameter_reg = {key:self.combination_builder(el, repeating.get(key, False)) for key, el in types.items()}
        if optional:
            func_regex = f"(?: {func_regex})?"
        func_regex = func_regex.format(**parameter_reg)
        group_mapper = list()
        for key, reg in parameter_reg.items():
            group_mapper.append({
                "regex":reg,
                "group":counter,
                "repeating":repeating.get(key,False),
                "needed": not optional,
                "implicite":False
            })
            counter += 1
        
        return (func_regex, group_mapper, counter)



    def combination_builder(self, types:tuple, repeating:bool = False)->str:
        comb = "|".join(self.regex_lookUp.get(t) for t in types)
        if repeating:
            return f"((?:(?:{comb})(?:, )?)+)"
        return f"({comb})"


In [21]:
f = FunctionBuilder()
testinput = [
    {"text":"load data from excel file {file_name}", "types":{"file_name":(DataTypes.STRING,)}},
    {"text":"from the sheet {sheet_name}", "types":{"sheet_name":(DataTypes.STRING,DataTypes.INT)}},
    {"text":"and set the index to the column {index}", "types":{"index":(DataTypes.STRING,)}},
]
test_str = 'load data from excel file "test.xlsx" from the sheet "test" and set the index to the column "testo"'   

reg, groups = f.build_function(*testinput)
print(reg)
print()
print(groups)
print("result")
m = re.fullmatch(reg, test_str)
print(m)
print("group tests")
for par in groups:
    print(m.group(par.get("group")))

load data from excel file ((?:(?:"[^"]+")(?:, )?)+)(?: from the sheet ((?:(?:"[^"]+"|-?\d+)(?:, )?)+))?(?: and set the index to the column ((?:(?:"[^"]+")(?:, )?)+))?

[{'regex': '((?:(?:"[^"]+")(?:, )?)+)', 'group': 1, 'repeating': (<DataTypes.STRING: 3>,), 'needed': True, 'implicite': False}, {'regex': '((?:(?:"[^"]+"|-?\\d+)(?:, )?)+)', 'group': 2, 'repeating': (<DataTypes.STRING: 3>, <DataTypes.INT: 1>), 'needed': False, 'implicite': False}, {'regex': '((?:(?:"[^"]+")(?:, )?)+)', 'group': 3, 'repeating': (<DataTypes.STRING: 3>,), 'needed': False, 'implicite': False}]
result
<re.Match object; span=(0, 99), match='load data from excel file "test.xlsx" from the sh>
group tests
"test.xlsx"
"test"
"testo"


In [22]:
print(f.build_function(*[
    {"text":"change row from {from} to {to}", 
    "types":{"from":(DataTypes.FLOAT, DataTypes.INT, DataTypes.STRING, DataTypes.BOOL, DataTypes.NONE), "to":(DataTypes.FLOAT, DataTypes.INT, DataTypes.STRING, DataTypes.BOOL, DataTypes.NONE)}, 
    "repeating":{"from":True, "to":True}
    },
    ]))

('change row from ((?:(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None)(?:, )?)+) to ((?:(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None)(?:, )?)+)', [{'regex': '((?:(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None)(?:, )?)+)', 'group': 1, 'repeating': (<DataTypes.FLOAT: 2>, <DataTypes.INT: 1>, <DataTypes.STRING: 3>, <DataTypes.BOOL: 4>, <DataTypes.NONE: 5>), 'needed': True, 'implicite': False}, {'regex': '((?:(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None)(?:, )?)+)', 'group': 2, 'repeating': (<DataTypes.FLOAT: 2>, <DataTypes.INT: 1>, <DataTypes.STRING: 3>, <DataTypes.BOOL: 4>, <DataTypes.NONE: 5>), 'needed': True, 'implicite': False}])


In [23]:
attrTest = {
    "text":"change row from {from} to {to}",
    "types":{
        "from":(DataTypes.ATTRMAP,),
        "to":(DataTypes.ATTRMAP,)},
    "repeating":{"from":True, "to":True}}
print(f.build_function(attrTest))

('change row from ((?:(?:(?:`[^`]+`)=(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None))(?:, )?)+) to ((?:(?:(?:`[^`]+`)=(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None))(?:, )?)+)', [{'regex': '((?:(?:(?:`[^`]+`)=(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None))(?:, )?)+)', 'group': 1, 'repeating': (<DataTypes.ATTRMAP: 8>,), 'needed': True, 'implicite': False}, {'regex': '((?:(?:(?:`[^`]+`)=(?:-?\\d+\\.\\d+|-?\\d+|"[^"]+"|True|False|None))(?:, )?)+)', 'group': 2, 'repeating': (<DataTypes.ATTRMAP: 8>,), 'needed': True, 'implicite': False}])


In [24]:
attr_controll = {
    "text":"a {map}",
    "types":{"map":(DataTypes.ATTRMAP,)},
    "repeating":{}
}
print(f.build_function(attr_controll)[0])

a ((?:(?:(?:`[^`]+`)=(?:-?\d+\.\d+|-?\d+|"[^"]+"|True|False|None))(?:, )?)+)


In [25]:
print(f.build_function(
    {
        "text": "delete data",
        "types":{},
        "repeating":{},
        "implicite":True
    },
    {
        "text": "but keep cols",
        "types":{},
        "repeating":{},
        "implicite":True
    },
))

('delete data(but keep cols)?', [{'regex': None, 'group': 1, 'repeating': False, 'needed': False, 'implicite': True}])
